In [9]:
import numpy as np 
import matplotlib.pyplot as plt
from qutip import *

<h2>
    <center>
        General Ramsey Spectroscopy
    </center>
</h2>

Suppose that we have a pulse with turn-on time much faster than the time scale of the system. According to [Physics Stack Exchange](https://physics.stackexchange.com/questions/226115/exact-meaning-of-pi-2-pulse) the pulse operates on the system as

$$
R_z(\theta) = \cos(\theta/2) \mathbb{I} - i \sin(\theta/2) \sigma_z
$$

where $\theta$ is the rotation angle of the qubit state and \mathbb{I} is the identity operator. 

In [34]:
class N_dipole_system:
    def __init__(self, N):
        # Number of atoms 
        self.N = N

        ### Hamiltonian ###
        self.H = 0

        ### annihilation operators ###
        idatom = qeye(2)  # identity operator
        sm = projection(2, 0, 1)  # |1><0| for an Ns-state system

        self.sm_list = []
        for i in range(N):
            op_list = [idatom] * N
            op_list[i] = sm
            self.sm_list.append(tensor(op_list))

        
        ### initial states ###
        self.ket_excited = ket( [1]*N, dim = [2] * N)
        self.ket_ground = ket( [0]*N, dim = [2] * N)

        # total spin operators
        self.id_tot = tensor([qeye(2)] * N)
        self.sm_tot = self.ket_ground * self.ket_excited.dag()
        self.sp_tot = self.sm_tot.dag()
        self.sz_tot = self.sp_tot * self.sm_tot - self.sm_tot * self.sp_tot
    
    def Rz(self, theta):
        return np.cos(theta/2)*self.id_tot - 1j*np.sin(theta/2)*self.sz_tot
    
    def pulse(self, psi, theta):
        return self.Rz(theta) * psi

    def evolve(self, psi0, tlist, e_ops, c_ops=[], store_states=True):
        return mesolve(self.H, psi0, tlist, c_ops=c_ops, e_ops=e_ops, store_states=store_states, store_final_state=True)

<h2>
    <center>
        Two non-interacting atoms with different frequencies
    </center>
</h2>

Suppose that we have two non-interacting atoms with different frequencies. The Hamiltonian of the system is

$$
H = \frac{\hbar \omega_1}{2} \sigma_1^\dagger \sigma_1 + \frac{\hbar \omega_2}{2} \sigma_2^\dagger \sigma_2
$$

where $\omega_1$ and $\omega_2$ are the frequencies of the two atoms. For simplicity, we assume that the decay time is much longer than the time scale of the experiment. 

In [36]:
sys = N_dipole_system(2)

omega_0 = -1/2
omega_1 = 1/2
sys.H = sum([
    omega_0 * sys.sm_list[0].dag() * sys.sm_list[0]
])

psi0 = sys.ket_ground
psi0 = sys.pulse(psi0, np.pi/2)
print(psi0)

tlist = np.linspace(0, 1, 100)
res = sys.evolve(psi0, tlist, [sys.sm_list[0].dag() * sys.sm_list[0]])
print(res.states)

# plot
plt.subplots(figsize=(10, 6))
plt.plot(tlist, res.expect[0], label="excited")
plt.xlabel('Time')
plt.ylabel('Occupation probability')

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.70710678+0.70710678j]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]]


TypeError: mesolve() got an unexpected keyword argument 'store_states'

In [27]:
res.states

[]